In [1]:
import polars as pl
from io import StringIO
import time
import pyfiglet

# Start the timer
start_time = time.time()

# Read and preprocess the original GTF file
file_path = r"C:\Users\DJ\Documents\PhD\COMS 6100\Final\Omyk_2.0.gtf"

# Read and preprocess the file to skip comments and ensure correct columns
with open(file_path, 'r') as infile:
    cleaned_data = "\n".join([
        line.strip() for line in infile
        if not line.startswith("#") and len(line.strip().split("\t")) == 9
    ])

# Use StringIO to create an in-memory file and read it directly into Polars
df = pl.read_csv(StringIO(cleaned_data), separator="\t", has_header=False).rename({
    "column_1": "seqname", "column_2": "source", "column_3": "feature",
    "column_4": "start", "column_5": "end", "column_6": "score",
    "column_7": "strand", "column_8": "frame", "column_9": "attribute"
})

# Filter and process the data
df_exons = (
    df.filter((pl.col("seqname").str.contains("Omy")) & (pl.col("feature") == "exon"))  # Filter seqname and feature
    .filter(pl.col("source").str.contains("mRNA"))  # Filter mRNA
    .with_columns([  # Extract gene_id and isoform
        pl.col("attribute").str.extract(r'gene_id "([a-zA-Z0-9_.]+)"').alias("gene_id"),
        pl.col("attribute").str.extract(r'transcript_id "([a-zA-Z0-9_.]+)"').alias("isoform")
    ])
    .filter(pl.col("gene_id").str.contains(r"[a-zA-Z]+"))  # Remove rows with numeric-only or missing gene_id
    .sort(["gene_id", "start", "end"])  # Sort by gene_id, start, and end
    .with_columns(  # Create start-end pair column
        (pl.col("start").cast(str) + "_" + pl.col("end").cast(str)).alias("startEnd_pair")
    )
    .with_columns(  # Identify duplicates
        pl.col("startEnd_pair").is_duplicated().alias("is_duplicate")
    )
    .filter(~pl.col("is_duplicate"))  # Remove duplicates
)

# Group and aggregate results
result_df = (
    df_exons.group_by("gene_id").agg([
        pl.col("isoform").unique().alias("isoforms"),  # Get unique isoforms
        pl.col("startEnd_pair").n_unique().alias("unique_exon_count")  # Count unique start-end pairs
    ])
    .with_columns(  # Convert isoforms to comma-separated string
        pl.col("isoforms")
        .list.eval(pl.element().sort())  # Sort isoforms
        .list.join(", ").alias("isoforms")  # Join into a string
    )
    .sort("gene_id")  # Sort final results by gene_id
)
result_df = result_df.rename({
    "gene_id": "Gene",
    "isoforms": "Isoforms",
    "unique_exon_count": "Unique Exon Count"
})
# Save the results to a csv file
output_file = r"C:\Users\DJ\Documents\PhD\COMS 6100\Final\unique_exons_summary_DJ.csv"
result_df.write_csv(output_file)

# Sanity check the results
pb3_row = result_df.filter(pl.col("Gene") == "PB.3")
pb3_exon_count = pb3_row[0, "Unique Exon Count"]
print(f"Unique exon count for PB.3: {pb3_exon_count}")

# End the timer
end_time = time.time()
print(f"Final result saved to {output_file}")
print(f"Script completed in {end_time - start_time:.2f} seconds")





Unique exon count for PB.3: 23
Final result saved to C:\Users\DJ\Documents\PhD\COMS 6100\Final\unique_exons_summary_DJ.csv
Script completed in 4.16 seconds
